In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
# Set default plotly theme
pio.templates.default = 'plotly_white'

from src.utilities import Matrix

from src.observables import *

SUFFIX = '.npy'
OMEGA = 0.5  # Interaction Strength

In [ ]:
# Files utilities

def file_dims(filename):
    return int(filename.split('_')[-3][1:])


def file_time(filename):
    return int(filename.split('_')[-2][1:])


def file_timedelta(filename):
    return float(filename.split('_')[-1][2:])


def check_file_metadata(filename, d, dt):
    dims = file_dims(filename)
    timedelta = file_timedelta(filename)
    return dims == d and timedelta == dt


def get_all_files(state, observable):
    files = [file for file in os.listdir(f'../objects/{state}') if file.endswith(SUFFIX) and file.startswith(f'rho_{observable}')]
    return files

def get_last_file(startswith, dims, state):
    all_files = [f.removesuffix(SUFFIX) for f in os.listdir(f'../objects/{state}') if f.startswith(f'{startswith}')]
    timedeltas = [file_timedelta(f) for f in all_files]
    max_times = {dt: max([file_time(f) for f in all_files if check_file_metadata(f, dims, dt)]) for dt in timedeltas}
    files = {dt: [f for f in all_files if check_file_metadata(f, dims, dt) and file_time(f) == max_times[dt]][0] for dt in timedeltas}
    return dict(sorted(files.items()))

def get_cov_of_dims(dims, state, filtered_keys=None) -> dict:
    files = get_last_file(f'rho_covariance_D{dims}', dims, state)
    if filtered_keys is not None:
        files = {k: files[k] for k in filtered_keys}
    return files

def get_heat_of_dims(dims, state, filtered_keys=None) -> dict:
    files = get_last_file(f'rho_heats_D{dims}', dims, state)
    if filtered_keys is not None:
        files = {k: files[k] for k in filtered_keys}
    return files

In [ ]:
def single_entropy(cov, system):
    d1, d2 = symplectic_eigenvalues(cov)
    if system == 1:
        x1 = d1 + 1/2
        x2 = d1 - 1/2
    elif system == 2:
        x1 = d2 + 1/2
        x2 = d2 - 1/2
    return x1 * np.log(x1) - x2 * np.log(x2)

In [ ]:
def get_observable(observable, covs, iterator):
    if observable == 'Entropy':
        func = symplectic_entropy
    elif observable == 'Mutual Information':
        func = mutual_information
    elif observable == 'Quantum Discord':
        func = gaussian_quantum_discord
    elif observable == 'Logarithmic Negativity':
        func = logarithmic_negativity
    else:
        raise ValueError(f'Observable {observable} not supported')

    arr = []
    for i in iterator:
        arr.append(func(covs[i]))
    return np.array(arr).real

# Effects of Interaction Time on the evolution

In [ ]:
observables = ['Temperatures', 'Entropies', 'Mutual Information', 'Quantum Discord', 'Logarithmic Negativity', 'Heat Flux']
df = pd.DataFrame(columns=observables)
state = 'coherent'
dims = 17
filtered_timedeltas = [0.05, 0.1, 1.0]
cov_files = get_cov_of_dims(dims, state, filtered_timedeltas)
heat_transfers = get_heat_of_dims(dims, state, filtered_timedeltas)
for timedelta, file in cov_files.items():
    dt_df = pd.DataFrame(columns=observables)
    cov_evolution = np.load(f'../objects/{state}/' + file + SUFFIX)
    heat_transfer = np.load(f'../objects/{state}/' + heat_transfers[timedelta] + SUFFIX).real
    print(f'{len(cov_evolution)} evolution steps with interaction time {timedelta}')

    iterator = np.linspace(0, len(cov_evolution) - 1, 2000).astype(int)
    dt_df['Time'] = [i * timedelta for i in iterator]
    dt_df['Interaction Time'] = timedelta
    dt_df['d1'] = np.array([symplectic_eigenvalues(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d2'] = np.array([symplectic_eigenvalues(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d1-'] = np.array([symplectic_eigenvalues_transposed(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d2-'] = np.array([symplectic_eigenvalues_transposed(cov_evolution[i])[1] for i in iterator]).real

    for obs in observables:
        if obs == 'Heat Flux':
            dt_df['J1'] = np.array([heat_transfer[i, 0] for i in iterator])
            dt_df['J2'] = np.array([heat_transfer[i, 1] for i in iterator])
            dt_df['Jc'] = np.array([heat_transfer[i, 2] for i in iterator])
        elif obs == 'Entropies':
            dt_df['S'] = np.array([symplectic_entropy(cov_evolution[i]) for i in iterator]).real
            dt_df['S1'] = np.array([single_entropy(cov_evolution[i], 2) for i in iterator]).real
            dt_df['S2'] = np.array([single_entropy(cov_evolution[i], 1) for i in iterator]).real
        elif obs == 'Temperatures':
            dt_df['N1'] = np.array([mean_photon_numbers(cov_evolution[i])[1] for i in iterator]).real
            dt_df['N2'] = np.array([mean_photon_numbers(cov_evolution[i])[0] for i in iterator]).real
        else:
            dt_df[obs] = get_observable(obs, cov_evolution, iterator)

    df = pd.concat([df, dt_df])
df.drop('Heat Flux', axis=1, inplace=True)
df.drop('Entropies', axis=1, inplace=True)
df.drop('Temperatures', axis=1, inplace=True)

In [ ]:
import qutip as qu

dm = qu.tensor(qu.coherent_dm(17, 1, method='analytic'), qu.coherent_dm(17, 1, method='analytic'))
operators = [
    qu.tensor(qu.position(17), qu.qeye(17)).full(),
    qu.tensor(qu.momentum(17), qu.qeye(17)).full(),
    qu.tensor(qu.qeye(17), qu.position(17)).full(),
    qu.tensor(qu.qeye(17), qu.momentum(17)).full(),
]
cv = covariance(dm.full(), operators)
a = qu.tensor(qu.destroy(17), qu.qeye(17)).full()
ad = qu.tensor(qu.create(17), qu.qeye(17)).full()
np.trace(dm.full() @ ad @ a)
np.trace(cv[0:2, 0:2])

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Interaction Time'], value_vars=['N1', 'N2'], var_name='System', value_name='Temperature')
fig_temp = px.line(df_melt, x='Time', y='Temperature',
                   color='Interaction Time', line_dash='System',
                   title='Mean Photon Numbers in the Systems',
                   labels={'Temperature': 'Mean Photon Number', 'Time': 'Time (a.u.)'})
fig_temp.update_xaxes(range=[0, 1500])
fig_temp.show()

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Interaction Time'], value_vars=['S1', 'S2'], var_name='System', value_name='Entropy')
fig_entropy = px.line(df_melt, x='Time', y='Entropy',
                   color='Interaction Time', line_dash='System',
                   title='Systems Entropies')
fig_entropy.update_xaxes(range=[0, 1500])
fig_entropy.show()

In [ ]:
fig_discord = px.line(df, x='Time', y='Quantum Discord', color='Interaction Time', title='Quantum Discord')
fig_discord.update_xaxes(range=[0, 1500])
fig_discord.show()

In [ ]:
fig_mutinf = px.line(df, x='Time', y='Mutual Information', color='Interaction Time', title='Mutual Information')
fig_mutinf.update_xaxes(range=[0, 1500])
fig_mutinf.show()

In [ ]:
fig_logneg = px.line(df, x='Time', y='Logarithmic Negativity', color='Interaction Time', title='Logarithmic Negativity')
fig_logneg.update_xaxes(range=[0, 1500])
fig_logneg.show()

In [ ]:
df_melt = df.loc[df['Time']>0].melt(id_vars=['Time', 'Interaction Time'], value_vars=['J1', 'J2', 'Jc'], var_name='Current', value_name='Heat')
fig_heat = px.line(df_melt, x='Time', y='Heat',
                   color='Interaction Time', line_dash='Current',
                   title='Heat Currents')
fig_heat.update_xaxes(range=[0, 200])
fig_heat.update_yaxes(range=[-0.005, 0.0001])
fig_heat.show()

## Symplectic Eigenvalues

In [ ]:
fig, ax = plt.subplots()
x=df.loc[df['Interaction Time'] == 1.0]['Time']
y1=df.loc[df['Interaction Time'] == 1.0]['d1']
y2=df.loc[df['Interaction Time'] == 1.0]['d1-']
ax.plot(x, y1, y2)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x=df.loc[df['Interaction Time'] == 1.0]['Time']
y1=df.loc[df['Interaction Time'] == 1.0]['d2']
y2=df.loc[df['Interaction Time'] == 1.0]['d2-']

ax.plot(x, y1, y2)
plt.show()

## Save DataFrame

In [ ]:
df.to_pickle(f'../objects/{state}/observables.pkl')